In [1]:
import numpy as np

In [4]:
import importlib
import dataImportFuctions

importlib.reload(dataImportFuctions)
from dataImportFuctions import main_data_dic_generator

## imprting all data of controls and patients for both parcellations

In [1]:
#for controls
controls_base_dir = '/Users/user/Int Dim NF1 WM/controls Schaeffer300_AAL3_timeseries_00301_Run1'
controls_file_pattern = 'Schaeffer300_AAL3_timeseries_{:03d}_Run1.mat'  # {:03d} will be replaced by numbers with leading zeros
#controls_index_list= [3,4,6,7,8,10,11,12,13,14,15,17,18,19]
controls_index_list= [1,2,3,4,6,7,8,10,11,12,13,14,15,17,18,19] #the original
N_c= len(controls_index_list)

#for patients
patients_base_dir = '/Users/user/Int Dim NF1 WM/patients Schaeffer300_AAL3_timeseries_00301_Run1'
patients_file_pattern = 'Schaeffer300_AAL3_timeseries_{:03d}01_Run1.mat'
#patients_index_list= [3,4,5,6,7,8,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,29,30,32,33]
patients_index_list= [1,2,3,4,5,6,7,8,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,29,30,32,33] #the original
N_p= len(patients_index_list)

In [5]:
#controls
#AAL3
controls_AAL3_main_dic= main_data_dic_generator(controls_base_dir,
                                           controls_file_pattern, 
                                           controls_index_list,
                                           'AAL3',
                                           True) #print_bool
#Schaeffer300                     
controls_Schaeffer300_main_dic= main_data_dic_generator(controls_base_dir,
                                           controls_file_pattern, 
                                           controls_index_list,
                                           'Schaeffer300',
                                           False) #print_bool

#patients
#AAL3
patients_AAL3_main_dic= main_data_dic_generator(patients_base_dir,
                                           patients_file_pattern, 
                                           patients_index_list,
                                           'AAL3',
                                           False) #print_bool
#Schaeffer300                                           
patients_Schaeffer300_main_dic= main_data_dic_generator(patients_base_dir,
                                           patients_file_pattern, 
                                           patients_index_list,
                                           'Schaeffer300',
                                           False) #print_bool                                    

dict_keys(['__header__', '__version__', '__globals__', 'AAL3', 'Schaeffer_300'])
key=  001
(144, 166)
dict_keys(['__header__', '__version__', '__globals__', 'AAL3', 'Schaeffer_300'])
key=  002
(144, 166)
dict_keys(['__header__', '__version__', '__globals__', 'AAL3', 'Schaeffer_300'])
key=  003
(144, 166)
dict_keys(['__header__', '__version__', '__globals__', 'AAL3', 'Schaeffer_300'])
key=  004
(144, 166)
dict_keys(['__header__', '__version__', '__globals__', 'AAL3', 'Schaeffer_300'])
key=  006
(144, 166)
dict_keys(['__header__', '__version__', '__globals__', 'AAL3', 'Schaeffer_300'])
key=  007
(144, 166)
dict_keys(['__header__', '__version__', '__globals__', 'AAL3', 'Schaeffer_300'])
key=  008
(144, 166)
dict_keys(['__header__', '__version__', '__globals__', 'AAL3', 'Schaeffer_300'])
key=  010
(144, 166)
dict_keys(['__header__', '__version__', '__globals__', 'AAL3', 'Schaeffer_300'])
key=  011
(144, 166)
dict_keys(['__header__', '__version__', '__globals__', 'AAL3', 'Schaeffer_300'])
k